# Wildfire Risk - Ingest
__Team 3 - Dave Friesen, John Chen, and Kyle Dalope__<br>
__ADS-508-02-SP23__<br><br>
__GitHub link: https://github.com/davefriesen/wildfire-risk__

In [2]:
__authors__ = ['Dave Friesen', 'John Chen', 'Kyle Dalope']
__contact__ = ['dfriesen@sandiego.edu', 'johnchen@sandiego.edu', 'kdalope@sandiego.edu']
__date__ = '2023-03-20'
__license__ = 'MIT'
__version__ = '1.0.1'

# Setup Basics

In [3]:
# Import basic libraries
import boto3
import sagemaker

# Import data access libraries
import pandas as pd
from profiler import profile, profile_cat
!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

# Import utility libraries
from IPython.core.display import display, HTML

In [4]:
# Establish session fundamentals
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.Session().client(service_name='sagemaker', region_name=region)

# Setup S3

In [5]:
# Set public path and store as SageMaker variable
s3_public_path = 's3://wildfire-risk/'
%store s3_public_path

# Set private path ("my bucket")
s3_private_path = 's3://{}/widfire-risk/data/'.format(bucket)
%store s3_private_path
print(s3_private_path)

Stored 's3_public_path' (str)
Stored 's3_private_path' (str)
s3://sagemaker-us-east-1-074876746575/widfire-risk/data/


In [6]:
# Get contents of Public S3 bucket (file source)
!aws s3 ls $s3_public_path

2023-03-24 02:49:22    8623294 conditions.csv
2023-03-18 23:31:25   10821874 fires.csv
2023-03-18 23:31:21   38317925 weather.csv


In [7]:
# Now copy public bucket-based data to private (local) bucket
!aws s3 cp --recursive $s3_public_path $s3_private_path'fires'/ --exclude '*' --include 'fires.csv'
!aws s3 cp --recursive $s3_public_path $s3_private_path'weather'/ --exclude '*' --include 'weather.csv'
!aws s3 cp --recursive $s3_public_path $s3_private_path'conditions'/ --exclude '*' --include 'conditions.csv'

copy: s3://wildfire-risk/fires.csv to s3://sagemaker-us-east-1-074876746575/widfire-risk/data/fires/fires.csv
copy: s3://wildfire-risk/weather.csv to s3://sagemaker-us-east-1-074876746575/widfire-risk/data/weather/weather.csv
copy: s3://wildfire-risk/conditions.csv to s3://sagemaker-us-east-1-074876746575/widfire-risk/data/conditions/conditions.csv


In [8]:
# List contents of private S3 bucket to confirm copy
!aws s3 ls $s3_private_path'fires'/
!aws s3 ls $s3_private_path'weather'/
!aws s3 ls $s3_private_path'conditions'/

2023-03-26 15:47:39   10821874 fires.csv
2023-03-26 15:47:41   38317925 weather.csv
2023-03-26 15:47:42    8623294 conditions.csv


# Load and Validate Data (*traditional Pandas - as check*)

In [9]:
# Load and check base data files from public S3 bucket
fires_df = pd.read_csv(s3_private_path+'fires/fires.csv')
weather_df = pd.read_csv(s3_private_path+'weather/weather.csv')
conditions_df = pd.read_csv(s3_private_path+'conditions/conditions.csv')
profile(fires_df)
profile(weather_df)
profile(conditions_df)

,Dtype,count,unique,na,na%,mean,std,min,max,skew(>=3),<v0.01,VIF(>=10),examples
ContainmentDateTime,object,35315,33108,53847,60.4,,,,,,,,2021/08/02 23:51:59+
ControlDateTime,object,32570,30401,56592,63.5,,,,,,,,2021/08/03 20:03:00+
DiscoveryAcres,float64,72195,348,16967,19.0,5.8,500.0,,115997.0,191.8,,,0.1__0.1__0.2__0.01_
EstimatedCostToDate,float64,2128,1040,87034,97.6,8193477.0,34440012.7,,800000000.0,13.1,,,nan__nan__nan__nan__
FinalAcres,float64,2631,345,86531,97.0,14.9,318.1,,13440.0,34.3,,,nan__nan__nan__nan__
FireBehaviorGeneral,object,1498,4,87664,98.3,,,,,,,,nan__nan__nan__nan__
FireBehaviorGeneral1,object,1223,16,87939,98.6,,,,,,,,nan__nan__nan__nan__
FireBehaviorGeneral2,object,1164,16,87998,98.7,,,,,,,,nan__nan__nan__nan__
FireBehaviorGeneral3,object,910,16,88252,99.0,,,,,,,,nan__nan__nan__nan__
FireCause,object,77967,4,11195,12.6,,,,,,,,Natural__Human__Huma


,Dtype,count,unique,na,na%,mean,std,min,max,skew(>=3),<v0.01,VIF(>=10),examples
STATION,object,175300,610,,,,,,,,,,USR0000CDEM__USR0000
DATE,object,175300,461,,,,,,,,,,2003-11__2006-04__20
LATITUDE,float64,175300,600,,,40.1,3.9,32.6,49.0,,,131.0,39.46__39.64__40.009
LONGITUDE,float64,175300,590,,,-118.0,6.3,-124.4,-82.4,,,266.0,-108.88__-105.4647__
ELEVATION,float64,175300,472,,,1214.0,734.3,,3694.2,,,,2181.5__2502.4__2115
NAME,object,175300,610,,,,,,,,,,"DEMAREE COLORADO, CO"
CDSD,float64,148117,11831,27183,15.5,182.8,315.5,,3218.1,3.1,,,477.7__0.0__168.6__5
CDSD_ATTRIBUTES,object,143784,1,31516,18.0,,,,,,,,U__U__U__U__nan
CLDD,float64,174374,4080,926,0.5,34.0,67.2,,601.0,,,23.0,0.0__0.0__0.0__224.8
CLDD_ATTRIBUTES,object,174374,6,926,0.5,,,,,,,,"2,U__,U__,U__,U__,U"


,Dtype,count,unique,na,na%,mean,std,min,max,skew(>=3),<v0.01,VIF(>=10),examples
INVYR,int64,120208,21,,,2008.4,6.9,1994.0,2019.0,,,25.2,2001__2008__2015__20
COND_STATUS_CD,int64,120208,5,,,1.8,1.0,1.0,5.0,,,10.1,2__2__1__2__2
MAPDEN,float64,48111,3,72097,60.0,1.0,0.1,1.0,3.0,8.5,,33.1,nan__nan__1.0__nan__
STDAGE,float64,50653,488,69555,57.9,93.5,95.6,,9999.0,23.7,,,nan__nan__75.0__nan_
STDSZCD,float64,52176,4,68032,56.6,1.5,1.0,1.0,5.0,,,,nan__nan__1.0__nan__
FLDSZCD,float64,48111,6,72097,60.0,2.7,1.0,,5.0,,,19.6,nan__nan__3.0__nan__
SITECLCD,float64,54682,7,65526,54.5,4.6,1.6,1.0,7.0,,,,nan__nan__4.0__nan__
SICOND,float64,45612,175,74596,62.1,90.7,26.9,7.0,192.0,,,,nan__nan__109.0__nan
STDORGCD,float64,48111,2,72097,60.0,0.2,0.4,,1.0,,,,nan__nan__0.0__nan__
SLOPE,float64,53696,138,66512,55.3,31.5,23.9,,155.0,,,,nan__nan__8.0__nan__


# Setup Athena Database

In [10]:
# Set Athena database name
database_name = 'dsoaws'

# Set S3 staging directory (temp directory for Athena queries)
s3_staging_dir = 's3://{0}/athena/staging'.format(bucket)

# Establish S3 connection
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

# Create Athena database
statement = 'CREATE DATABASE IF NOT EXISTS {}'.format(database_name)
pd.read_sql(statement, conn)

# Verify database creation
statement = 'SHOW DATABASES'
df_show = pd.read_sql(statement, conn)

if database_name in df_show.values:
    ingest_create_athena_db_passed = True
%store ingest_create_athena_db_passed    

df_show.head(5)

Stored 'ingest_create_athena_db_passed' (bool)


,database_name
0,default
1,dsoaws


## Register fires.csv as Athena table

In [11]:
fires_csv = 'fires.csv'
fires_tb = 'fires'

# Force table creation
statement = """DROP TABLE IF EXISTS {}.{}""".format(
    database_name, fires_tb
)    
pd.read_sql(statement, conn)

# Create table
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
    ContainmentDateTime string,
    ControlDateTime string,
    DiscoveryAcres double,
    EstimatedCostToDate double,
    FinalAcres string,
    FireBehaviorGeneral string,
    FireBehaviorGeneral1 string,
    FireBehaviorGeneral2 string,
    FireBehaviorGeneral3 string,
    FireCause string,
    FireCauseGeneral string,
    FireCauseSpecific string,
    FireDiscoveryDateTime string,
    FireOutDateTime string,
    GACC string,
    IncidentName string,
    IncidentShortDescription string,
    InitialLatitude double,
    InitialLongitude double,
    IsFireCauseInvestigated double,
    IsTrespass double,
    POOCity string,
    POOState string,
    PredominantFuelModel string,
    PrimaryFuelModel string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, fires_tb, s3_private_path+fires_tb+'/'
)
print(statement)

pd.read_sql(statement, conn)

CREATE EXTERNAL TABLE IF NOT EXISTS dsoaws.fires(
    ContainmentDateTime string,
    ControlDateTime string,
    DiscoveryAcres double,
    EstimatedCostToDate double,
    FinalAcres string,
    FireBehaviorGeneral string,
    FireBehaviorGeneral1 string,
    FireBehaviorGeneral2 string,
    FireBehaviorGeneral3 string,
    FireCause string,
    FireCauseGeneral string,
    FireCauseSpecific string,
    FireDiscoveryDateTime string,
    FireOutDateTime string,
    GACC string,
    IncidentName string,
    IncidentShortDescription string,
    InitialLatitude double,
    InitialLongitude double,
    IsFireCauseInvestigated double,
    IsTrespass double,
    POOCity string,
    POOState string,
    PredominantFuelModel string,
    PrimaryFuelModel string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-074876746575/widfire-risk/data/fires/'
TBLPROPERTIES ('skip.header.line.count'='1')


""


## Register weather.csv as Athena table

In [12]:
weather_csv = 'weather.csv'
weather_tb = 'weather'

# Force table creation
statement = """DROP TABLE IF EXISTS {}.{}""".format(
    database_name, weather_tb
)    
pd.read_sql(statement, conn)

# Create table
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
    Station string,
    WDate string,
    Latitude double,
    Longitude double,
    Elevation double,
    Name string,
    CDSD double,
    CDSD_attributes string,
    CLDD double,
    CLDD_attributes string,
    DT00 double,
    DT00_attributes string,
    DT32 double,
    DT32_attributes string,
    DX32 double,
    DX32_attributes string,
    DX70 double,
    DX70_attributes string,
    DX90 double,
    DX90_attributes string,
    EMNT double,
    EMNT_attributes string,
    EMXT double,
    EMXT_attributes string,
    HDSD double,
    HDSD_attributes string,
    HTDD double,
    HTDD_attributes string,
    TAVG double,
    TAVG_attributes string,
    TMAX double,
    TMAX_attributes string,
    TMIN double,
    TMIN_attributes string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, weather_tb, s3_private_path+weather_tb+'/'
)
print(statement)

pd.read_sql(statement, conn)

CREATE EXTERNAL TABLE IF NOT EXISTS dsoaws.weather(
    Station string,
    WDate string,
    Latitude double,
    Longitude double,
    Elevation double,
    Name string,
    CDSD double,
    CDSD_attributes string,
    CLDD double,
    CLDD_attributes string,
    DT00 double,
    DT00_attributes string,
    DT32 double,
    DT32_attributes string,
    DX32 double,
    DX32_attributes string,
    DX70 double,
    DX70_attributes string,
    DX90 double,
    DX90_attributes string,
    EMNT double,
    EMNT_attributes string,
    EMXT double,
    EMXT_attributes string,
    HDSD double,
    HDSD_attributes string,
    HTDD double,
    HTDD_attributes string,
    TAVG double,
    TAVG_attributes string,
    TMAX double,
    TMAX_attributes string,
    TMIN double,
    TMIN_attributes string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-074876746575/widfire-risk/data/weather/'
TBLPROPERTIES ('skip.header.line.count'='1')


""


## Register conditions.csv as Athena table

In [13]:
conditions_csv = 'conditions.csv'
conditions_tb = 'conditions'

# Force table creation
statement = """DROP TABLE IF EXISTS {}.{}""".format(
    database_name, conditions_tb
)    
pd.read_sql(statement, conn)

# Create table
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
    PLT_CN integer,
    INVYR integer,
    COND_STATUS_CD integer,
    MAPDEN double,
    STDAGE double,
    STDSZCD double,
    FLDSZCD double,
    SITECLCD double,
    SICOND double,
    STDORGCD double,
    SLOPE double,
    PHYSCLCD double,
    GSSTKCD double,
    DSTRBCD1 double,
    TRTCD1 double,
    PRESNFCD double,
    FLDAGE double,
    CARBON_DOWN_DEAD double,
    CARBON_LITTER double,
    CARBON_SOIL_ORG double,
    CARBON_STANDING_DEAD double,
    CARBON_UNDERSTORY_AG double,
    CARBON_UNDERSTORY_BG double,
    STATE string,
    WATERCD double,
    LAT double,
    LON double
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, conditions_tb, s3_private_path+conditions_tb+'/'
)
print(statement)

pd.read_sql(statement, conn)

CREATE EXTERNAL TABLE IF NOT EXISTS dsoaws.conditions(
    PLT_CN integer,
    INVYR integer,
    COND_STATUS_CD integer,
    MAPDEN double,
    STDAGE double,
    STDSZCD double,
    FLDSZCD double,
    SITECLCD double,
    SICOND double,
    STDORGCD double,
    SLOPE double,
    PHYSCLCD double,
    GSSTKCD double,
    DSTRBCD1 double,
    TRTCD1 double,
    PRESNFCD double,
    FLDAGE double,
    CARBON_DOWN_DEAD double,
    CARBON_LITTER double,
    CARBON_SOIL_ORG double,
    CARBON_STANDING_DEAD double,
    CARBON_UNDERSTORY_AG double,
    CARBON_UNDERSTORY_BG double,
    STATE string,
    WATERCD double,
    LAT double,
    LON double
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-074876746575/widfire-risk/data/conditions/'
TBLPROPERTIES ('skip.header.line.count'='1')


""


## Verify table creation

In [14]:
statement = "SHOW TABLES in {}".format(database_name)
df_show = pd.read_sql(statement, conn)

if [fires_tb, weather_tb] in df_show.values:
    ingest_create_athena_table_passed = True
%store ingest_create_athena_table_passed

df_show.head(5)

Stored 'ingest_create_athena_table_passed' (bool)


,tab_name
0,amazon_reviews_tsv
1,conditions
2,fires
3,fires_parquet
4,weather


## Run sample queries

In [15]:
statement = """SELECT * FROM {}.{}
    WHERE FireCause = 'Natural' LIMIT 50""".format(
    database_name, fires_tb
)
print(statement)

df = pd.read_sql(statement, conn)
df.head(5)

SELECT * FROM dsoaws.fires
    WHERE FireCause = 'Natural' LIMIT 50


,containmentdatetime,controldatetime,discoveryacres,estimatedcosttodate,finalacres,firebehaviorgeneral,firebehaviorgeneral1,firebehaviorgeneral2,firebehaviorgeneral3,firecause,...,incidentname,incidentshortdescription,initiallatitude,initiallongitude,isfirecauseinvestigated,istrespass,poocity,poostate,predominantfuelmodel,primaryfuelmodel
0,,,3.0,NaN,,,,,,Natural,...,DL OAK NOB,,43.28994,-122.5181,NaN,0.0,,US-OR,,
1,,,NaN,0.0,,Active,,,,Natural,...,O'Shea Cr.,Tiller,NaN,NaN,NaN,NaN,Tiller,US-OR,,Timber (Grass and Understory)
2,,,1.5,NaN,,,,,,Natural,...,Little Grayback,,42.19094,-122.4942,NaN,0.0,,US-OR,,
3,,,0.1,NaN,,,,,,Natural,...,Dudley Mountain,,42.63354,-122.5394,NaN,0.0,,US-OR,,
4,,,NaN,NaN,,,,,,Natural,...,MITCHELL,,NaN,NaN,NaN,0.0,,US-CA,,


In [16]:
statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, weather_tb
)
print(statement)

df = pd.read_sql(statement, conn)
df.head(5)

SELECT * FROM dsoaws.weather LIMIT 10


,station,wdate,latitude,longitude,elevation,name,cdsd,cdsd_attributes,cldd,cldd_attributes,...,hdsd,hdsd_attributes,htdd,htdd_attributes,tavg,tavg_attributes,tmax,tmax_attributes,tmin,tmin_attributes
0,USR0000CABS,1995-01,40.3983,-122.8333,579.1,"""ARBUCKLE BASIN CALIFORNIA",None,0.0,None,0.0,...,0.0,"""",None,-6.1,None,U,7.0,"""",11.1,""""
1,USR0000CABS,1995-02,40.3983,-122.8333,579.1,"""ARBUCKLE BASIN CALIFORNIA",None,0.0,None,0.0,...,0.0,"""",None,1.1,None,U,14.0,"""",21.7,""""
2,USR0000CABS,1995-03,40.3983,-122.8333,579.1,"""ARBUCKLE BASIN CALIFORNIA",None,0.0,None,0.0,...,0.0,"""",None,-3.9,None,U,6.0,"+""",18.9,""""
3,USR0000CABS,1995-04,40.3983,-122.8333,579.1,"""ARBUCKLE BASIN CALIFORNIA",None,0.3,None,0.3,...,0.0,"""",None,3.3,None,U,15.0,"""",22.2,""""
4,USR0000CABS,1995-05,40.3983,-122.8333,579.1,"""ARBUCKLE BASIN CALIFORNIA",None,23.1,None,22.8,...,0.0,"""",None,3.3,None,U,12.0,"""",29.4,""""


In [17]:
statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, conditions_tb
)
print(statement)

df = pd.read_sql(statement, conn)
df.head(5)

SELECT * FROM dsoaws.conditions LIMIT 10


,plt_cn,invyr,cond_status_cd,mapden,stdage,stdszcd,fldszcd,siteclcd,sicond,stdorgcd,...,carbon_down_dead,carbon_litter,carbon_soil_org,carbon_standing_dead,carbon_understory_ag,carbon_understory_bg,state,watercd,lat,lon
0,2019,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None
1,2019,1,1.0,30.0,2.0,2.0,5.0,103.0,1.0,40.0,...,8.104615,18.434625,0.927575,2.274576,0.252731,None,None,None,None,None
2,2019,1,1.0,NaN,3.0,1.0,4.0,106.0,0.0,35.0,...,14.140916,12.305077,0.404615,1.797825,0.199758,None,None,None,None,None
3,2019,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None
4,2019,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None


# Setup Parquet

In [18]:
# Set Parquet basics
s3_parquet_dir = 's3://{0}/parquet'.format(bucket)

## Create Parquet file from fires.csv

In [19]:
fires_tb_parquet = 'fires_parquet'

# Force table creation
statement = """DROP TABLE IF EXISTS {}.{}""".format(
    database_name, fires_tb_parquet
)    
pd.read_sql(statement, conn)

# SKIP PARTITIONING INITIALLY: WITH (format = 'PARQUET', external_location = '{}', partitioned_by = ARRAY['firecause']) AS
# Create table
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (format = 'PARQUET', external_location = '{}') AS
SELECT
    ContainmentDateTime,
    ControlDateTime,
    DiscoveryAcres,
    EstimatedCostToDate,
    FinalAcres,
    FireBehaviorGeneral,
    FireBehaviorGeneral1,
    FireBehaviorGeneral2,
    FireBehaviorGeneral3,
    FireCause,
    FireCauseGeneral,
    FireCauseSpecific,
    FireDiscoveryDateTime,
    FireOutDateTime,
    GACC,
    IncidentName,
    IncidentShortDescription,
    InitialLatitude,
    InitialLongitude,
    IsFireCauseInvestigated,
    IsTrespass,
    POOCity,
    POOState,
    PredominantFuelModel,
    PrimaryFuelModel
FROM {}.{}""".format(
    database_name, fires_tb_parquet, s3_private_path+fires_tb_parquet+'/', database_name, fires_tb
)
print(statement)

pd.read_sql(statement, conn)

CREATE TABLE IF NOT EXISTS dsoaws.fires_parquet
WITH (format = 'PARQUET', external_location = 's3://sagemaker-us-east-1-074876746575/widfire-risk/data/fires_parquet/') AS
SELECT
    ContainmentDateTime,
    ControlDateTime,
    DiscoveryAcres,
    EstimatedCostToDate,
    FinalAcres,
    FireBehaviorGeneral,
    FireBehaviorGeneral1,
    FireBehaviorGeneral2,
    FireBehaviorGeneral3,
    FireCause,
    FireCauseGeneral,
    FireCauseSpecific,
    FireDiscoveryDateTime,
    FireOutDateTime,
    GACC,
    IncidentName,
    IncidentShortDescription,
    InitialLatitude,
    InitialLongitude,
    IsFireCauseInvestigated,
    IsTrespass,
    POOCity,
    POOState,
    PredominantFuelModel,
    PrimaryFuelModel
FROM dsoaws.fires


DatabaseError: Execution failed on sql: CREATE TABLE IF NOT EXISTS dsoaws.fires_parquet
WITH (format = 'PARQUET', external_location = 's3://sagemaker-us-east-1-074876746575/widfire-risk/data/fires_parquet/') AS
SELECT
    ContainmentDateTime,
    ControlDateTime,
    DiscoveryAcres,
    EstimatedCostToDate,
    FinalAcres,
    FireBehaviorGeneral,
    FireBehaviorGeneral1,
    FireBehaviorGeneral2,
    FireBehaviorGeneral3,
    FireCause,
    FireCauseGeneral,
    FireCauseSpecific,
    FireDiscoveryDateTime,
    FireOutDateTime,
    GACC,
    IncidentName,
    IncidentShortDescription,
    InitialLatitude,
    InitialLongitude,
    IsFireCauseInvestigated,
    IsTrespass,
    POOCity,
    POOState,
    PredominantFuelModel,
    PrimaryFuelModel
FROM dsoaws.fires
HIVE_PATH_ALREADY_EXISTS: Target directory for table 'dsoaws.fires_parquet' already exists: s3://sagemaker-us-east-1-074876746575/widfire-risk/data/fires_parquet. You may need to manually clean the data at location 's3://sagem

In [ ]:
# Load partitions
statement = "MSCK REPAIR TABLE {}.{}".format(database_name, fires_tb_parquet)
print(statement)

df = pd.read_sql(statement, conn)
df.head(5)

In [ ]:
# Show partitions
statement = "SHOW PARTITIONS {}.{}".format(database_name, fires_tb_parquet)
print(statement)

#df_partitions = pd.read_sql(statement, conn)
#df_partitions.head(5)

In [ ]:
# Show tables
statement = "SHOW TABLES in {}".format(database_name)
df_tables = pd.read_sql(statement, conn)

if fires_tb_parquet in df_tables.values:
    ingest_create_athena_table_parquet_passed = True
%store ingest_create_athena_table_parquet_passed

df_tables.head(5)

In [ ]:
# Run sample query
statement = """SELECT * FROM {}.{}
    WHERE FireCause = 'Natural' LIMIT 50""".format(
    database_name, fires_tb_parquet
)
print(statement)

df = pd.read_sql(statement, conn)
df.head(5)

# Store Variables and Close Session

In [ ]:
# Store variables for subsequent notebooks
%store

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}